Generates random bar plots from 6 different bar plot combinations, applies elastic transform and removes corners in order to make it more real. Then,it inserts these new plots into the given slides in the source directory. For each plot, it also saves the information about which coordinates it has been put and image size.

In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import imageio, elasticdeform
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from PIL import Image
import glob
import itertools
import random
from faker import Faker
fake = Faker()
csfont = {'fontname':'Comic Sans MS'}

In [2]:
# Takes and image and applies random transformation w.r.t. given alpha and sigma values.
# alpha – scale of transformation for each dimension, larger values have more deformation
# sigma – Gaussian window of deformation for each dimension smaller values have more localised deformation
def elastic_transform(image, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    assert len(image.shape)==2

    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), indexing='ij')
    indices = np.reshape(x+dx, (-1, 1)), np.reshape(y+dy, (-1, 1))
    
    return map_coordinates(image, indices, order=1).reshape(shape)

In [3]:
# Removes randomly corners
# From corners list, 75% of corners in a given image are removed.
# For each corner, it paints a circle in determined color below
# Takes image as an input and returns an image 
def remove_corners(image):
    img=cv2.imread(image,0)
    corners = cv2.goodFeaturesToTrack(img, 100, 0.01, 10)
    corners = np.int0(corners)
    
    for corner in corners:
        n = random.randint(1,4)
        if(n != 3):
            x,y = corner.ravel()
            cv2.circle(img,(x,y),3,255,-1)
    return img

In [4]:
# Creates random bar plot
def get_random_plot():
    words = [] # X label
    performance = [] # Y label
    number = random.randint(2,6) #How many bars
    width = random.uniform(0.6, 0.8) #Each bars' width
    linewidth = random.uniform(4.5, 5.5) #Each bars' line width
    #Finds fakes words w.r.t. given randomly number of bars
    for _ in range(number):
      words.append(fake.word())

    y_pos = np.arange(len(words))
    #Assign to each word a random value for plotting
    for _ in range(number):
      performance.append(random.randint(1,41))
    #Draw the bar plot #fill=False
    plt.bar(y_pos, performance, width, align='center', alpha=0.9, linewidth = linewidth, color='black')
    plt.xticks([])
    plt.yticks([])
    #plt.title(fake.sentence(),**csfont)
    #plt.show()
   
#add_image_to_pptx('test.pptx', 3, 'image.jpg', Inches(1.5), Inches(1.5))

In [5]:
# Plots 3 bars side by side
# Takes x to determine image name and distortation values alpha and sigma
# Creates 3 subplots and determines some features
# Returns the new image
def three_side(x, alpha=5, sigma=3):
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.tight_layout()
    
    ax = plt.subplot(1, 3, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()

    ax = plt.subplot(1, 3, 2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()

    ax = plt.subplot(1, 3, 3)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()
    image_name = 'image' + str(x) + '.png' # Saves image name
    plt.savefig(image_name, dpi=100) # Image is saved by reducing 75% of its size
    plt.close()

    img = cv2.imread(image_name,0)
    img = remove_corners(image_name) #Apply remove corners
    img = elastic_transform(img, alpha, sigma, np.random) #Apply elastic distortation
    #cv2.imshow('Corner',img)
    return img

In [6]:
# Plots 3 bars below
# Takes x to determine image name and distortation values alpha and sigma
# Creates 3 subplots and determines some features
# Returns the new image
def three_below(x, alpha=5, sigma=3):
    fig, axes = plt.subplots(nrows=3, ncols=1)
    fig.tight_layout()
    ax = plt.subplot(3, 1, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()

    ax = plt.subplot(3, 1, 2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()

    ax = plt.subplot(3, 1, 3)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()
    image_name = 'image' + str(x) + '.png' # Saves image name
    plt.savefig(image_name, dpi=100) # Image is saved by reducing 75% of its size
    plt.close()
    img = cv2.imread(image_name,0)
    img = remove_corners(image_name) #Apply remove corners
    img = elastic_transform(img, alpha, sigma, np.random) #Apply elastic distortation
    return img

In [7]:
# Plots two bars side by side
# Takes x to determine image name and distortation values alpha and sigma
# Creates 2 subplots and determines some features
# Returns the new image
def two_side(x, alpha=5, sigma=3):
    fig, axes = plt.subplots(nrows=1, ncols=2)
    fig.tight_layout()
    ax = plt.subplot(1, 2, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()
    
    ax = plt.subplot(1, 2, 2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()
    image_name = 'image' + str(x) + '.png' # Saves image name
    plt.savefig(image_name, dpi=100) # Image is saved by reducing 75% of its size
    plt.close()
    img = cv2.imread(image_name,0)
    img = remove_corners(image_name) #Apply remove corners
    img = elastic_transform(img, alpha, sigma, np.random) #Apply elastic distortation
    return img

In [8]:
# Plots two bars below
# Takes x to determine image name and distortation values alpha and sigma
# Creates 2 subplots and determines some features
# Returns the new image
def two_below(x, alpha=4, sigma=2):
    fig, axes = plt.subplots(nrows=2, ncols=1)
    fig.tight_layout()
    ax = plt.subplot(2, 1, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()

    ax = plt.subplot(2, 1, 2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()
    image_name = 'image' + str(x) + '.png' # Saves image name
    plt.savefig(image_name, dpi=100) # Image is saved by reducing 75% of its size
    plt.close()
    img = cv2.imread(image_name,0)
    img = remove_corners(image_name) #Apply remove corners
    img = elastic_transform(img, alpha, sigma, np.random) #Apply elastic distortation
    return img

In [9]:
# Plots four bars in square shape
# Takes x to determine image name and distortation values alpha and sigma
# Creates 4 subplots and determines some features
# Returns the new image
def four_square(x, alpha=3.5, sigma=2):
    fig, axes = plt.subplots(nrows=2, ncols=2)
    fig.tight_layout()
    ax = plt.subplot(2, 2, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()

    ax = plt.subplot(2, 2, 2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()

    ax = plt.subplot(2, 2, 3)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(6)
    ax.spines['left'].set_linewidth(6)
    get_random_plot()
    
    ax = plt.subplot(2, 2, 4)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()
    image_name = 'image' + str(x) + '.png' # Saves image name
    plt.savefig(image_name, dpi=100) # Image is saved by reducing 75% of its size
    plt.close()
    img = cv2.imread(image_name,0)
    img = remove_corners(image_name) #Apply remove corners
    img = elastic_transform(img, alpha, sigma, np.random) #Apply elastic distortation
    return img

In [10]:
# Plots a single bar 
# Takes x to determine image name and distortation values alpha and sigma
# Creates 1 subplots and determines some features
# Returns the new image
def single(x, alpha=6, sigma=3):
    
    ax = plt.subplot(1, 1, 1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(5)
    ax.spines['left'].set_linewidth(5)
    get_random_plot()
    image_name = 'image' + str(x) + '.png' # Saves image name
    plt.savefig(image_name, dpi=100) # Image is saved by reducing 75% of its size
    plt.close()
    img = cv2.imread(image_name,0)
    img = remove_corners(image_name) #Apply remove corners
    img = elastic_transform(img, alpha, sigma, np.random) #Apply elastic distortation
    return img

In [19]:
# Creates n plottings in a randomly selected type 
# Takes single argument to determine the number of images
def plot_creator(numberOfImages):
    prev = 0 # Previous random number
    for x in range(numberOfImages):
        n = random.randint(1,6)
        # Take new number until it is different from the previous random number
        while(n == prev):
            n = random.randint(1,6)
        if(n == 1):
            img = three_side(x)
        elif(n == 2):
            img = three_below(x)
        elif(n == 3):
            img = two_side(x)
        elif(n == 4):
            img = two_below(x)
        elif(n == 5):
            img = four_square(x)
        else:
            img = single(x)
            
        prev = n
        image_name = 'image' + str(x) + '.png' #Image saved name
        cv2.imwrite(image_name,255-img) #Saves as a negative color
        crop_image(image_name) #Crops images to remove borders
        img = cv2.imread(image_name,0)
        cv2.destroyAllWindows()

In [12]:
# Removes borders from the input image and saves 
def crop_image(image_name):
    img = cv2.imread(image_name,0)
    #img = cv2.resize(img,(100,200))
    cropped = img[2:98, 2:148] #146x96 pixel crops borders
    backtorgb = cv2.cvtColor(cropped,cv2.COLOR_GRAY2RGB)
    cv2.imwrite(image_name,backtorgb)
    #cv2.imwrite(image_name,cropped)

In [13]:
# Randomly select coordinates and plot is pasted to a destination and named as ith image.
def distrubute_images(plot, destination, i): 
    x = random.randint(15,95) 
    y = random.randint(18,135)

    #f = open("./generated/results.txt","a+")
    try: 
        #Relative Path 
        #Image on which we want to paste 
        img = Image.open(destination)  

        #Relative Path 
        #Image which we want to paste 
        img2 = Image.open(plot)  
        img.paste(img2, (x, y)) # margins of top left corner 
        # Write coordinates of pasted image to txt
        f = open('./generated/' + str(i) + '.txt',"a+")
        f.write('0 ' + str((x+73)/256.) + ' ' + str((y+48)/256.) + ' ' + str(96/256.) + ' ' + str(146/256.) + '\n')
        #Saved in the same relative location 
        img.save('./generated/' + str(i) + ".jpg") 

    except IOError: 
        pass
    #return mylist 

In [46]:
#Given extent type, finds all files and saves into list
def get_file_names(exts):
    fnames = [glob.glob(ext) for ext in exts]
    fnames = list(itertools.chain.from_iterable(fnames))
    return fnames

In [47]:
#Creates data. 
#rang is how many plotting do you want to crete
#multiplier is for each source image, how mant times are plottings pasted 
#Creates number of images in source * multiplier images
def insert_slides(rang = 10, multiplier = 2):
    #plot_creator(rang) #Creates plot
    exts = ["./source/*.jpg"]
    res = get_file_names(exts)
    namer = 0
    for j in range(multiplier):
        for imx in res:
            image_name = 'image' + str(random.randint(0,rang)) + '.png'
            distrubute_images(image_name, imx, namer)
            namer = namer+1
        

In [48]:
insert_slides(10000,2)

In [20]:
plot_creator(30)